### **Consumo y Demanda de Energía en Sistemas de Potencia**

#### **Objeto**

Poner en práctica los conceptos y temas vistos en el curso propedéutico, a fin de plantear propuestas de aproximación para el análisis del comportamiento del consumo y la demanda de Energía Eléctrica (EE) en un sistema de potencia.

#### **Contenido:**
+ Definiciones
+ Planteamientos de aproximación propuestos:
+ Pronóstico de la demanda y consumo
+ Cálculo de energía consumida
+ Análisis estadístico del perfil de demanda y consumo
+ Modelado del fenómeno (demanda/consumo de EE)
+ Interpolación/Extrapolación
+ Integrador de funciones
+ Series de tiempo
+ Conclusiones y trabajos futuros

#### **Planteamientos**

**1 - Pronóstico de la demanda y consumo**
Dada una estadística de valores de demanda y consumo (de septiembre de 2017 a julio de 2020 –BD de trabajo) se pretende caracterizar un modelo polinomial para estimar los valores de demanda/consumo de los próximos meses.

**¿Por qué?**
El pronóstico del consumo y la demanda de energía eléctrica es una actividad esencial para la operación y planeación de los sistemas de potencia, a fin de garantizar su correcta operación en los diferentes horizontes de tiempo.

**2 - Cálculo del consumo de energía**
La información de consumo recabada en la BD de trabajo se calcula utilizando promedios para cada intervalo de 1 hora. Se desea calcular la energía de el mismo periodo utilizando funciones aproximadas que se ajusten a la forma de la curva de demanda y compararlas.

**¿Por qué?**
Es importante definir la granularidad de la información que se puede obtener de los sistemas de registros. Considerar periodos menores a 1 hora podría significar una gran cantidad de recursos para su almacenamiento y procesamiento. Por eso es importante definir hasta qué grado se pueden obtener estimaciones adecuadas con diferentes espaciamiento en las muestras. 

**3 - Análisis estadístico del perfil de demanda/consumo.
La forma que tiene una curva de demanda presenta ciertos patrones que sugieren comportamientos repetitivos en diferentes horizontes de tiempo (día, semana, año). Se plantea analizar el fenómeno de demanda de energía y consumo como una serie de tiempo para obtener más información de su comportamiento.

**¿Por qué?**
Debido a las variaciones que presenta el fenómeno de demanda y consumo de EE es de interés dar una representación estadística para identificar patrones que se puedan predecir y ayuden en la toma de decisiones en la operación y planeación del sistema de potencia.

